# OPERA L2-RTC Validation 

OPERA L2-RTC Validation is performed on OPERA bursts that are merged into the shape of the source Sentinel-1 scenes.

Each validation workflow is a collection of notebooks in this repository. They may be run piecemeal, on individual scenes, but it is time consuming.

**This notebook makes it easy to run validation modules in bulk, on all of the data submitted for validation.** 

1. Select a validation module
1. Select a site and orbital path combination
1. Download non-OPERA data used in validation
1. Run a bulk validation script
    1. Downloads and mosaics the bursts
    1. Runs validation workflows
    
After running validation modules from this notebook, the input and output validation data can be found in the `../OPERA_L2-RTC_CalVal` directory.

In [ ]:
from getpass import getpass
import os
from pathlib import Path
import subprocess
from typing import Union
from zipfile import ZipFile 

import earthaccess
from opensarlab_lib import work_dir

import ipywidgets as widgets
from ipywidgets import Layout

## 1. Select Validation Module

In [ ]:
calval_types = [
    "Absolute Location Evaluation (ALE)",
    "Flattening",
    "Relative Location Evaluation (RLE)",
]

calval_module = widgets.RadioButtons(
    options=calval_types,
    description='',
    disabled=False,
    layout=Layout(min_width="1000px")
    )

print("Select an OPERA-RTC validation analysis to perform.")
display(calval_module)

## 2. Select a validation site and orbital path

In [ ]:
if "Absolute" in calval_module.value:
    
    sites = [
        "California, Ascending Orbit, Orbital Path 64",
    ]
    script = "bulk_papermill_OPERA_RTC_ALE.py"
elif "RLE" in calval_module.value:
    sites = [
        "Delta Junction, Ascending Orbit, Path 94",
        "Delta Junction, Descending Orbit, Path 160",
        "Brazil, Descending Orbit, Path 170",
    ]
    script = "bulk_papermill_OPERA_RTC_RLE.py"
elif calval_module.value == 'Flattening':
    sites = [
        "Vermont, Ascending Orbit, Path 135",
        "Vermont, Descending Orbit, Path 113",
        "Delta Junction, Descending Orbit, Path 160",
        "Brazil, Descending Orbit, Path 39",
    ]
    script = "bulk_papermill_OPERA_RTC_flattening.py"
    
else:
    raise Exception(f"{calval_module.value} analysis option not yet added to this notebook")
    
site_options = widgets.RadioButtons(
    options=sites,
    description='',
    disabled=False,
    layout=Layout(min_width="1000px")
    )

print("Select a site/path for the validation analysis.")
display(site_options)

## 3. Select whether to download and validate data or validate previously downloaded data

In [ ]:
download_mosaic = [
    'Download, mosaic, and validate OPERA sample burst data',
    'Validate previously downloaded and mosaicked data'
]

download_options = widgets.RadioButtons(
    options=download_mosaic,
    description='',
    disabled=False,
    layout=Layout(min_width="1000px")
    )

print("Download and mosaic OPERA sample burst data or validate previously prepared mosaics?")
display(download_options)

if "RLE" in calval_module.value:
    print("\nSave or delete intermediary data produced during validation?")
    delete_options = widgets.RadioButtons(
        options=['Save intermediary data', 'Delete intermediary data'],
        description='',
        disabled=False,
        layout=Layout(min_width="1000px")
        )
    display(delete_options)

## 4. Download [Copernicus Global Land Cover (100m)](https://lcviewer.vito.be/download) and [NISAR Calibration Corner Reflector](https://uavsar.jpl.nasa.gov/cgi-bin/calibration-nisar.pl) data.  

In [ ]:
if "Download" in download_options.value:
    calval_pth = Path.cwd().parent / "OPERA_L2-RTC_CalVal"
    zip_pth = Path.cwd().parent / "OPERA_L2-RTC_CalVal.zip"

    if not zip_pth.exists() and not calval_pth.exists():
        download_cmd = f'wget --no-check-certificate --no-proxy "https://asf-opera-rtc-calval.s3.us-west-2.amazonaws.com/OPERA_L2-RTC_LC_CR_Input_Data/OPERA_L2-RTC_CalVal.zip" -q --show-progress -P {Path.cwd().parent}'
        subprocess.run(download_cmd, stdout=None, stderr=subprocess.PIPE, shell=True)
    if zip_pth.exists() and not calval_pth.exists():
        with ZipFile(zip_pth, 'r') as z:  
            z.extractall(path=Path.cwd().parent)    

## Log into EarthAccess with your EarthData Login credentials

- This will prompt for EDL credentials if they are not already present in a `.netrc` file
- Saves your EDL credentials to a `.netrc` file if not already present

In [ ]:
earthaccess.login(persist=True)

## 5. Run bulk validation script

In [ ]:
download = '--skip_download' if 'Validate' in download_options.value else ''
delete = '--delete_intermediary' if ('RLE' in calval_module.value and 'delete_options' in locals() and 'Delete' in delete_options.value) else ''
script_dir = Path.cwd()/"bulk_validation_scripts"
with work_dir(script_dir):
    site = site_options.value.split(',')[0]
    orbital_path = site_options.value.split(' ')[-1]
    subprocess.run([f'python {script} --site "{site}" --orbital_path {orbital_path} {download} {delete}'], shell=True)